# Running functions on IPython engines

## Start cluster / Load the module

In [ ]:
!ipcluster start -n 4 --daemonize

In [9]:
import ipyparallel

## Create a client instance, used to connect the controller to the remote engines

In [10]:
rc = ipyparallel.Client(profile='default')
nengines = len(rc)
nengines

4

In [11]:
rc.ids

[0, 1, 2, 3]

## Create a `DirectView`

A `DirectView` is created by list access to a client instance:

In [12]:
dvall  = rc[:]
print(dvall)

<DirectView [0, 1, 2, 3]>


In [13]:
type(dvall)

ipyparallel.client.view.DirectView

## Running Python functions on the engines
### Blocking

In blocking mode the `apply` function waits until all function calls are completed.

In [21]:
import os
dvall.block = True
hostnames = dvall.apply(socket.gethostname)
print(hostnames)

['schleppy-pro.local', 'schleppy-pro.local', 'schleppy-pro.local', 'schleppy-pro.local']


`apply_sync` is blocking

In [24]:
dvall.block = False
pids = dvall.apply_sync(os.getpid)
print(pids)

[13698, 13699, 13700, 13701]


In [23]:
for i in range(nengines):
    istr = '{:02d}'.format(i)  # returns a 2-digit string whose value is i
    pstr = str(pids[i])
    hstr = str(hostnames[i])
    msg = 'Engine '+istr+':   pid = '+pstr+';  hostname ='+hstr
    print(msg)

Engine 00:   pid = 13698;  hostname =schleppy-pro.local
Engine 01:   pid = 13699;  hostname =schleppy-pro.local
Engine 02:   pid = 13700;  hostname =schleppy-pro.local
Engine 03:   pid = 13701;  hostname =schleppy-pro.local


### Non blocking

In [38]:
def mypid():
    import time, os
    time.sleep(10)
    pid = os.getpid()
    return pid

Note the import to make sure the module is imported on the engines.

In [45]:
dvall.block = False
res = dvall.apply(mypid)
res

<AsyncResult: mypid>

#### Checking if result is ready

In [46]:
res.ready()

False

#### Blocking to get the result

In [47]:
pids = res.get()
pids

[13698, 13699, 13700, 13701]

In [ ]:
!ipcluster stop